In [ ]:
import sys
sys.path.append('/data/ECS/PythonTutorial/')
import log
log.topic_id = '23_VeriAnalizi-pandas-2'

# Pandas II

In [ ]:
import cx_Oracle
import os
import pandas as pd

def connect_db(user_name=None, password=None):
    try:
        with open('/data/home/alperayd/oracle.txt','r') as f:
            r = f.read()
            user_name = r.split(';')[0]
            password = r.split(';')[1]
        
        os.environ['ORACLE_HOME'] = '/data/app/oracle/product/12.2.0/client_1'
        host = "pgarsnb1rw.fw.garanti.com.tr:4520/pgarsnb1b"
        #host_dwh  = 'pgardwhrw:4520/pgardwh'
        #host_qgar = 'qgardwhapp.fw.garanti.com.tr:4520/QGARDWH'
        
        conn = cx_Oracle.connect(user_name, password, host, encoding='UTF-8')
        return conn
    except Exception as ex:
        print(f"bağlantı sağlanamadı, \nHata: {ex}")
        return None

    
conn = connect_db()
sql = """
    select * from pgaredm.v_cnf_customer_hst where record_Date=trunc(sysdate-1) and CUSTOMER_TP_CODE='TA'
"""
df = pd.read_sql(sql, conn)
display(df)

## lambda fonksiyonları hatırlatma

In [ ]:
def format_sube(x):
    return f"{x:06d}"

format_sube(123)

In [ ]:
y = lambda x: f"{x:06d}"

y(123)

## fillna()

In [ ]:
df.MAIN_BRANCH_NUM.fillna(0)#.astype(int).apply(format_sube)

## astype(int)

In [ ]:
df.MAIN_BRANCH_NUM.fillna(0).astype(int)

## apply()

In [ ]:
df.MAIN_BRANCH_NUM.fillna(0).astype(int).apply(lambda x: f"{x:06d}")

In [ ]:
df['SUBE_KODU_STR'] = df.MAIN_BRANCH_NUM.fillna(0).astype(int).apply(lambda x: f"{x:06d}")

In [ ]:
df.head()

### Örnek

In [ ]:
def dort_digit_sube_kodu(x):
    str(x)[-4:]
    x = str(x)[-4:]
    x = int(x)
    return f"{x:04d}"

dort_digit_sube_kodu(22)

In [ ]:
df.MAIN_BRANCH_NUM.fillna(0).astype(int).apply(dort_digit_sube_kodu)

### concat vs apply

In [ ]:
a = []
for i in df.MAIN_BRANCH_NUM.fillna(0).astype(int):
    a.append(dort_digit_sube_kodu(i))

df2 = pd.concat([df.MAIN_BRANCH_NUM.fillna(0).astype(int), 
           pd.Series(a)
          ] ,axis=1)
display(df2.head())

In [ ]:
df.MAIN_BRANCH_NUM.fillna(0).astype(int).apply(lambda x: dort_digit_sube_kodu(x))

In [ ]:
df['YENI_KOLON'] = df.MAIN_BRANCH_NUM.fillna(0).astype(int).apply(lambda x: dort_digit_sube_kodu(x))
df.head()

## apply: birden fazla kolon işlemi

In [ ]:
df.head()

In [ ]:
bolge_sube = df.fillna(0).apply(lambda x: f"{x.MAIN_BRANCH_NUM }" + '-' + f"{x.CUSTOMER_TP_CODE }", axis=1)
bolge_sube

In [ ]:
df

In [ ]:
df_yeni = pd.concat([df, bolge_sube], axis=1)
df_yeni

## assign

DataFrame'e yeni bir kolon ekler

### sabit değer ekleme

In [ ]:
df.assign(YENI_KOLON=1).head()

### fonksiyon ile ekleme

In [ ]:
df.IP_ID.max()

In [ ]:
df.assign(YENI_KOLON2 = lambda x: x.IP_ID.max()).head()

### apply fonksiyonu ile yeni kolon ekleme

In [ ]:
df.fillna(0).apply(lambda x: f"{x.MAIN_BRANCH_NUM }"+'-'+f"{x.CUSTOMER_TP_CODE }", axis=1).head()

In [ ]:
df = df.assign(YENI_KOLON3 = df.fillna(0).apply(lambda x: f"{x.MAIN_BRANCH_NUM }"+'-'+f"{x.CUSTOMER_TP_CODE }", axis=1))
df.head()

### assign - lambda function

In [ ]:
df.MAIN_BRANCH_NUM.fillna(0).astype(int) / df.SOURCE_SYSTEM_ID

In [ ]:
df = df.assign(IP_ORAN = lambda x: x.IP_ID / x.IP_ID.max())
display(df)

## groupby

### agg()

In [ ]:
df.columns[:50]

In [ ]:
dfg = df.groupby(['CUSTOMER_STATUS_CODE']).agg({'IP_ID': ['size','max','min'], #size: count
                                          'MAIN_BRANCH_NUM': ['size'] # len, count
                                         })
display(dfg)

In [ ]:
dfg.index

In [ ]:
dfg.loc['A', ('MAIN_BRANCH_NUM','size')]

### groupby: Series !önemli

In [ ]:
def get_location(x,y):
    return googapi.reverse_geocode(x,y)

In [ ]:
pd.Series({'A':1,'B':2,'C':3})

In [ ]:
df.groupby(['CUSTOMER_STATUS_CODE']).apply(lambda x: pd.Series({})

In [ ]:
df.groupby(['CUSTOMER_STATUS_CODE']).apply(lambda x: pd.Series({
    'Müşteri Adet': len(x.IP_ID),
    'Max': x.IP_ID.max(),
    'min': x.IP_ID.min(),
    'Şube Adet': len(x.MAIN_BRANCH_NUM),
    'yeni': len(x[x.IP_ID<1000])
}))

In [ ]:
def group_by_fonksiyonum(x):
    ret = pd.Series({
        'Müşteri Adet': len(x.IP_ID),
        'Max': x.IP_ID.max(),
        'Min': x.IP_ID.min(),
        'Şube Adet': len(x.MAIN_BRANCH_NUM),
        'yeni': len(x[x.IP_ID<1000])
    })
    return ret

In [ ]:
dfg = df.groupby(['CUSTOMER_STATUS_CODE']).apply(group_by_fonksiyonum)
display(dfg)

In [ ]:
dfg.loc['A']['Max']

In [ ]:
dfg.loc['A', ['Max','Min']]

### groupby içerisinde fonksiyon çalıştırma

In [ ]:
def karekok_al(ip_id):
    return ip_id**0.5
    
df.groupby(['CUSTOMER_STATUS_CODE']).apply(lambda x: pd.Series({
    'Müşteri Adet': len(x.IP_ID),
    'Max': x.IP_ID.max(),
    'Min': x.IP_ID.min(),
    'Şube Adet': len(x.MAIN_BRANCH_NUM),
    'yeni': len(x[x.IP_ID<1000]),
    'Fonksiyon': karekok_al(x.IP_ID).sort_values().values[:3]
}))

## pivot_table !önemli

In [ ]:
dft = pd.pivot_table(df, 
                     index=['MAIN_BRANCH_NUM'], 
                     columns=['CUSTOMER_STATUS_CODE'], 
                     values='IP_ID', aggfunc='count').fillna(0)
display(dft)

In [ ]:
pd.set_option('max_columns',30)

dft = pd.pivot_table(df, 
                     index=['MAIN_BRANCH_NUM'], 
                     columns=['CUSTOMER_STATUS_CODE'], 
                     values=['IP_ID'], aggfunc=['count','max']).fillna(0)
display(dft)

## sort_values

In [ ]:
df.sort_values(['CUSTOMER_STATUS_ID', 'IP_ID'], ascending=[False, True])[['CUSTOMER_STATUS_ID', 'IP_ID']]

## çoklu işlemler

In [ ]:
dft.fillna(0) \
    .astype(str) \
    .assign(
        Toplam=lambda x: x.sum(axis=1)
    ) \
    .sort_values(['Toplam'], ascending=[False]) \
    .head(10)

## transform

Oracle Analytic Functions:

count(IP_ID) over(partition by CUSTOMER_STATUS_ID)

In [ ]:
df.groupby(['CUSTOMER_STATUS_ID'])["IP_ID"].count()

In [ ]:
df = df.assign(STATUS_COUNT = df.groupby(['CUSTOMER_STATUS_ID'])['IP_ID'].transform('count'))

df[['IP_ID','CUSTOMER_STATUS_ID','STATUS_COUNT']]

## merge

In [ ]:
sql = """
    select MAIN_BRANCH_NUM, CUSTOMER_TP_CODE, count(*) 
    from pgaredm.v_cnf_customer_hst
    where RECORD_DATE=trunc(sysdate-1)
    group by MAIN_BRANCH_NUM, CUSTOMER_TP_CODE
"""
df2 = pd.read_sql(sql, connect_db())
df2.head()

In [ ]:
pd.merge(left=df[['MAIN_BRANCH_NUM','CUSTOMER_TP_CODE','IP_ID','CUSTOMER_STATUS_DESC']], 
         right=df2, 
         how='left', 
         left_on=['MAIN_BRANCH_NUM','CUSTOMER_TP_CODE'],
         right_on=['MAIN_BRANCH_NUM','CUSTOMER_TP_CODE'])

## to_excel

In [ ]:
import os
os.getcwd()

In [ ]:
sube_kodu = '119'
dft = df[(df.MAIN_BRANCH_NUM==sube_kodu)]
dft.to_excel(f"output_{sube_kodu}_alperayd.xlsx", index=False)

## pandas_profiling

In [ ]:
import pandas_profiling as pp
pp.ProfileReport(df.sample(1000))